<a href="https://colab.research.google.com/github/nngsam/Deep-Learning-/blob/main/W4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, MNIST, FashionMNIST
from torch.utils.data import DataLoader
from torchvision import models


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data augmentation

In [19]:
# Normalize và thêm các transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.RandomResizedCrop(size=28, scale=(0.8, 1.0)),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip()
])



In [20]:
# Load the FashionMNIST dataset
batch_size = 64
train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
# Defnied VGG provided
class MiniVGG(nn.Module):
    def __init__(self, num_classes= 10):
        super(MiniVGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels= 1, out_channels= 64, kernel_size= (3,3), stride= (1,1), padding= 1),
            nn.ReLU(),
            nn.Conv2d(in_channels= 64, out_channels= 64, kernel_size= (3,3), stride=(1,1), padding= 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= (2,2), stride= (2,2)),

            nn.Conv2d(in_channels= 64, out_channels= 128, kernel_size= (3,3), stride= (1,1), padding= 1),
            nn.ReLU(),
            nn.Conv2d(in_channels= 128, out_channels= 128, kernel_size= (3,3), stride=(1,1), padding= 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= (2,2), stride= (2,2)),

            nn.Conv2d(in_channels= 128, out_channels= 256, kernel_size= (3,3), stride= (1,1), padding= 1),
            nn.ReLU(),
            nn.Conv2d(in_channels= 256, out_channels= 256, kernel_size= (3,3), stride=(1,1), padding= 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= (2,2), stride= (2,2)))

        self.classifier = nn.Linear(256 * 3 * 3, 10)
        nn.init.normal_(self.classifier.weight, 0, 0.01)
        nn.init.constant_(self.classifier.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## Fintuning

In [23]:
# Load pretrained models on CIFAR-10 and MNIST
cifar10_model = MiniVGG().to(device)
cifar10_model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/cifar10_mini_vgg.pth", map_location=torch.device('cpu')))
for param in cifar10_model.parameters():
    param.requires_grad = False
for param in cifar10_model.classifier.parameters():
    param.requires_grad = True

mnist_model = MiniVGG().to(device)
mnist_model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/mnist_mini_vgg.pth", map_location=torch.device('cpu')))
for param in mnist_model.parameters():
    param.requires_grad = False
for param in mnist_model.classifier.parameters():
    param.requires_grad = True

In [24]:
scratch_model = MiniVGG().to(device)

In [25]:
scratch_model

MiniVGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Linear(in_features=2304, out_features=10, bias=True)
)

In [26]:
cifar10_model

MiniVGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Linear(in_features=2304, out_features=10, bias=True)
)

In [27]:
mnist_model

MiniVGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Linear(in_features=2304, out_features=10, bias=True)
)

In [30]:
# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)

# Set the batch size of the model
#model.batch_size = 3

for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = scratch_model(images)
        #loss.requires_grad = True
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        #loss.requires_grad = True
        optimizer.step()

        if (i + 1) % 100 == 0:
            print("Epoch: {}/5, Step: {}/{}, Loss: {:.4f}".format(
                epoch + 1, i + 1, len(train_loader), loss.item()
            ))

scratch_model.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = images.size(0)
        outputs = scratch_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    accuracy = total_correct / total_samples

    print(f'Learning Rate: 0.001, Accuracy: {accuracy}')

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: 1/5, Step: 100/938, Loss: 2.3014
Epoch: 1/5, Step: 200/938, Loss: 2.2977
Epoch: 1/5, Step: 300/938, Loss: 2.2802
Epoch: 1/5, Step: 400/938, Loss: 2.2377
Epoch: 1/5, Step: 500/938, Loss: 2.1053
Epoch: 1/5, Step: 600/938, Loss: 1.5728
Epoch: 1/5, Step: 700/938, Loss: 1.0841
Epoch: 1/5, Step: 800/938, Loss: 0.9977
Epoch: 1/5, Step: 900/938, Loss: 0.9802
Epoch: 2/5, Step: 100/938, Loss: 0.8069
Epoch: 2/5, Step: 200/938, Loss: 0.7696
Epoch: 2/5, Step: 300/938, Loss: 0.9956
Epoch: 2/5, Step: 400/938, Loss: 0.7657
Epoch: 2/5, Step: 500/938, Loss: 0.7863
Epoch: 2/5, Step: 600/938, Loss: 0.8890
Epoch: 2/5, Step: 700/938, Loss: 0.6647
Epoch: 2/5, Step: 800/938, Loss: 0.6409
Epoch: 2/5, Step: 900/938, Loss: 0.5834
Epoch: 3/5, Step: 100/938, Loss: 0.8001
Epoch: 3/5, Step: 200/938, Loss: 0.7006
Epoch: 3/5, Step: 300/938, Loss: 0.8928
Epoch: 3/5, Step: 400/938, Loss: 0.7827
Epoch: 3/5, Step: 500/938, Loss: 0.7259
Epoch: 3/5, Step: 600/938, Loss: 0.9950
Epoch: 3/5, Step: 700/938, Loss: 0.8946


In [31]:
# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cifar10_model.parameters(), lr=0.001, momentum=0.9)

# Set the batch size of the model
#model.batch_size = 3

for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = cifar10_model(images)
        #loss.requires_grad = True
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        #loss.requires_grad = True
        optimizer.step()

        if (i + 1) % 100 == 0:
            print("Epoch: {}/5, Step: {}/{}, Loss: {:.4f}".format(
                epoch + 1, i + 1, len(train_loader), loss.item()
            ))

cifar10_model.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = images.size(0)
        outputs = cifar10_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    accuracy = total_correct / total_samples

    print(f'Learning Rate: 0.001, Accuracy: {accuracy}')

In [ ]:
# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mnist_model.parameters(), lr=0.001, momentum=0.9)

# Set the batch size of the model
#model.batch_size = 3

for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = mnist_model(images)
        #loss.requires_grad = True
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        #loss.requires_grad = True
        optimizer.step()

        if (i + 1) % 100 == 0:
            print("Epoch: {}/5, Step: {}/{}, Loss: {:.4f}".format(
                epoch + 1, i + 1, len(train_loader), loss.item()
            ))

mnist_model.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = images.size(0)
        outputs = mnist_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    accuracy = total_correct / total_samples

    print(f'Learning Rate: 0.001, Accuracy: {accuracy}')

In [ ]:
# Test the model on FashionMNIST
# Test the models
def test_model(model, name):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy of {name}: {accuracy:.2f}%')

test_model(cifar10_model, "CIFAR-10 Model")
test_model(mnist_model, "MNIST Model")
test_model(scratch_model, "Scratch Model")

##  Feature Extraction

In [ ]:
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

train_nodes, eval_nodes = get_graph_node_names(scratch_model)

In [ ]:
train_nodes

In [ ]:
create_feature_extractor(scratch_model, train_return_nodes= train_nodes, eval_return_nodes= eval_nodes)

In [ ]:
scratch_model.features[0].weight